### Assignment 2: DES Simulation

In [25]:
import numpy as np
import simpy
import random
from scipy.stats import t
from scipy.stats import ttest_ind

In [56]:
# Simulation parameters
RANDOM_SEED = 42
SIM_TIME = 10000  # Simulation time
ARRIVAL_RATE = 2
SERVICE_RATE = 1
NUM_SERVERS = [1, 2, 4]
NUM_REPLICATIONS = 30  # Number of replications for each configuration


def service(env, service_rate):
    """Simulate the service of a job by a server."""
    service_time = random.expovariate(service_rate)
    yield env.timeout(service_time)


def process_job(env, servers, service_rate, waiting_time):
    """Handle the arrival and processing of a job."""
    arrival_time = env.now
    with servers.request() as request:
        yield request  # Wait for a server to be available
        wait_time = env.now - arrival_time
        waiting_time.append(wait_time)  # Record the waiting time for the job
        yield env.process(service(env, service_rate))


def arrival_process_fifo(env, arrival_rate, servers, service_rate, waiting_time):
    """Generate new jobs that arrive with an exponential interarrival time."""
    while True:
        interarrival_time = random.expovariate(arrival_rate)
        yield env.timeout(interarrival_time)
        env.process(process_job(env, servers, service_rate, waiting_time))


def run_simulation(num_servers, arrival_rate, service_rate, sim_time, seed):
    """Run the simulation for a given number of servers."""
    random.seed(seed)  # Ensure different random seeds for each replication
    
    # Create the simulation environment
    env = simpy.Environment()

    # Create the server resource
    servers = simpy.Resource(env, capacity=num_servers)
    
    # List to store waiting times
    waiting_time = []
    
    # Start the arrival process
    env.process(arrival_process_fifo(env, arrival_rate, servers, service_rate, waiting_time))
    
    # Run the simulation
    env.run(until=sim_time)
    
    # Return the average waiting time
    return np.mean(waiting_time)


def compute_confidence_interval(data, confidence=0.95):
    """Compute the confidence interval for a dataset."""
    mean = np.mean(data)
    std_error = np.std(data, ddof=1) / np.sqrt(len(data))  # Standard error of the mean
    margin = std_error * 1.96  # Approximation for 95% CI
    return mean, mean - margin, mean + margin


if __name__ == '__main__':
    print(f"Simulation Time: {SIM_TIME}")
    print(f"Service Rate (μ): {SERVICE_RATE}")
    print(f"Arrival Rate (λ): {ARRIVAL_RATE}")
    print(f"Number of Replications: {NUM_REPLICATIONS}")
    
    # Store results for each server configuration
    all_replications = {}

    for num_servers in NUM_SERVERS:
        replications = []
        
        for i in range(NUM_REPLICATIONS):
            seed = RANDOM_SEED + i  # Use a different seed for each replication
            avg_waiting_time = run_simulation(num_servers, ARRIVAL_RATE, SERVICE_RATE, SIM_TIME, seed)
            replications.append(avg_waiting_time)
        
        # Compute confidence interval
        mean, lower_bound, upper_bound = compute_confidence_interval(replications)
        
        print(f"\nNumber of Servers: {num_servers}")
        print(f"Mean Waiting Time: {mean:.2f}")
        print(f"95% Confidence Interval: ({lower_bound:.2f}, {upper_bound:.2f})")
        
        # Store replications for this server count
        all_replications[num_servers] = replications

    # Perform pairwise t-tests
    for i, num_servers1 in enumerate(NUM_SERVERS):
        for j, num_servers2 in enumerate(NUM_SERVERS):
            if i < j:  # Only compare each pair once
                data1 = all_replications[num_servers1]
                data2 = all_replications[num_servers2]
                
                # Perform t-test
                t_stat, p_value = ttest_ind(data1, data2, equal_var=False)  # Welch's t-test
                
                print(f"\nT-test between {num_servers1} servers and {num_servers2} servers:")
                print(f"T-statistic: {t_stat:.4f}")
                print(f"P-value: {p_value:.4f}")
                if p_value < 0.05:
                    print("Result: Significant difference (reject null hypothesis).")
                else:
                    print("Result: No significant difference (fail to reject null hypothesis).")


Simulation Time: 10000
Service Rate (μ): 1
Arrival Rate (λ): 2
Number of Replications: 30

Number of Servers: 1
Mean Waiting Time: 2496.08
95% Confidence Interval: (2482.39, 2509.78)

Number of Servers: 2
Mean Waiting Time: 43.68
95% Confidence Interval: (35.14, 52.22)

Number of Servers: 4
Mean Waiting Time: 0.09
95% Confidence Interval: (0.09, 0.09)

T-test between 1 servers and 2 servers:
T-statistic: 297.7708
P-value: 0.0000
Result: Significant difference (reject null hypothesis).

T-test between 1 servers and 4 servers:
T-statistic: 357.1641
P-value: 0.0000
Result: Significant difference (reject null hypothesis).

T-test between 2 servers and 4 servers:
T-statistic: 10.0020
P-value: 0.0000
Result: Significant difference (reject null hypothesis).


In [61]:
import simpy
import numpy as np
from scipy.stats import ttest_ind
from scipy.stats import t

# Simulation Parameters
RANDOM_SEED = 42
SIM_TIME = 10000  # Simulation time
ARRIVAL_RATE = 2
SERVICE_RATE = 1
NUM_SERVERS = [1]
NUM_REPLICATIONS = 30  # Number of replications for each configuration


def process_jobs(env, queue, waiting_time):
    """Process jobs based on shortest job first (SJF) using a PriorityStore."""
    while True:
        # Get the next job (shortest service time) from the queue
        priority, (service_time, arrival_time, job_id) = yield queue.get()

        # Record waiting time
        wait_time = env.now - arrival_time
        waiting_time.append(wait_time)

        # Simulate the service time
        yield env.timeout(service_time)


def arrival_process(env, queue, arrival_rate, service_rate, np_random):
    """Generate new jobs and insert them into a PriorityStore."""
    while True:
        interarrival_time = np_random.exponential(1.0 / arrival_rate)
        yield env.timeout(interarrival_time)

        # Generate a random service time
        service_time = np_random.exponential(1.0 / service_rate)
        arrival_time = env.now

        job_id = np_random.randint(0, int(1e6))  # Unique job ID
        job = (service_time, arrival_time, job_id)
        # Put the job into the queue with priority equal to service_time
        yield queue.put((service_time, job))


def arrival_process_fifo(env, servers, arrival_rate, service_rate, waiting_time, np_random):
    """Generate new jobs and process them in FIFO order."""
    while True:
        interarrival_time = np_random.exponential(1.0 / arrival_rate)
        yield env.timeout(interarrival_time)

        # Generate a random service time
        service_time = np_random.exponential(1.0 / service_rate)
        arrival_time = env.now

        def service(env, service_time, arrival_time):
            with servers.request() as request:
                # Wait for a server to be available
                yield request

                # Record waiting time
                wait_time = env.now - arrival_time
                waiting_time.append(wait_time)

                # Simulate the service time
                yield env.timeout(service_time)

        # Start the service process
        env.process(service(env, service_time, arrival_time))


def run_simulation(num_servers, arrival_rate, service_rate, sim_time, seed, sjfs=False):
    """Run the simulation with FIFO or Shortest Job First (SJF) scheduling."""
    np_random = np.random.RandomState(seed)
    env = simpy.Environment()

    # List to store waiting times
    waiting_time = []

    if sjfs:
        # Use PriorityStore for SJF scheduling
        queue = simpy.PriorityStore(env)
        env.process(arrival_process(env, queue, arrival_rate, service_rate, np_random))

        # Start as many server processes as there are servers
        for _ in range(num_servers):
            env.process(process_jobs(env, queue, waiting_time))
    else:
        # Create a resource to represent the servers
        servers = simpy.Resource(env, capacity=num_servers)
        env.process(arrival_process_fifo(env, servers, arrival_rate, service_rate, waiting_time, np_random))

    # Run the simulation
    env.run(until=sim_time)

    # Return the average waiting time
    return np.mean(waiting_time)


def compute_confidence_interval(data, confidence=0.95):
    """Compute the confidence interval for a dataset."""
    n = len(data)
    mean = np.mean(data)
    std_err = np.std(data, ddof=1) / np.sqrt(n)  # Standard error of the mean
    h = std_err * t.ppf((1 + confidence) / 2., n - 1)  # t critical value
    return mean, mean - h, mean + h


if __name__ == '__main__':
    print(f"Simulation Time: {SIM_TIME}")
    print(f"Service Rate (μ): {SERVICE_RATE}")
    print(f"Arrival Rate (λ): {ARRIVAL_RATE}")
    print(f"Number of Replications: {NUM_REPLICATIONS}")

    # Store results for each server configuration
    all_replications = {}

    for num_servers in NUM_SERVERS:
        replications = []

        for i in range(NUM_REPLICATIONS):
            seed = RANDOM_SEED + i  # Use a different seed for each replication
            avg_waiting_time = run_simulation(num_servers, ARRIVAL_RATE, SERVICE_RATE, SIM_TIME, seed)
            replications.append(avg_waiting_time)

        # Compute confidence interval
        mean, lower_bound, upper_bound = compute_confidence_interval(replications)

        print(f"\nNumber of Servers: {num_servers}")
        print(f"Mean Waiting Time: {mean:.2f}")
        print(f"95% Confidence Interval: ({lower_bound:.2f}, {upper_bound:.2f})")

        # Store replications for this server count
        all_replications[num_servers] = replications

    # Perform pairwise t-tests
    for i, num_servers1 in enumerate(NUM_SERVERS):
        for j, num_servers2 in enumerate(NUM_SERVERS):
            if i < j:  # Only compare each pair once
                data1 = all_replications[num_servers1]
                data2 = all_replications[num_servers2]

                # Perform t-test
                t_stat, p_value = ttest_ind(data1, data2, equal_var=False)  # Welch's t-test

                print(f"\nT-test between {num_servers1} servers and {num_servers2} servers:")
                print(f"T-statistic: {t_stat:.4f}")
                print(f"P-value: {p_value:.4f}")
                if p_value < 0.05:
                    print("Result: Significant difference (reject null hypothesis).")
                else:
                    print("Result: No significant difference (fail to reject null hypothesis).")

    # Now compare with M/M/1 with SJFS
    print("\n--- M/M/1 Queue with Shortest Job First (SJFS) ---")
    sjfs_replications = []

    for i in range(NUM_REPLICATIONS):
        seed = RANDOM_SEED + i
        avg_waiting_time = run_simulation(1, ARRIVAL_RATE, SERVICE_RATE, SIM_TIME, seed, sjfs=True)
        sjfs_replications.append(avg_waiting_time)

    mean_sjfs, lower_sjfs, upper_sjfs = compute_confidence_interval(sjfs_replications)

    print(f"Mean Waiting Time (M/M/1 with SJFS): {mean_sjfs:.2f}")
    print(f"95% Confidence Interval (M/M/1 with SJFS): ({lower_sjfs:.2f}, {upper_sjfs:.2f})")


Simulation Time: 10000
Service Rate (μ): 1
Arrival Rate (λ): 2
Number of Replications: 30

Number of Servers: 1
Mean Waiting Time: 2491.58
95% Confidence Interval: (2474.86, 2508.31)

--- M/M/1 Queue with Shortest Job First (SJFS) ---
Mean Waiting Time (M/M/1 with SJFS): 14.85
95% Confidence Interval (M/M/1 with SJFS): (13.44, 16.25)


In [66]:
NUM_SERVERS = [1, 3, 2, 4, 5]

def arrival_process_md(env, servers, arrival_rate, service_rate, waiting_time, np_random):
    """Generate new jobs with deterministic service times."""
    while True:
        # Generate interarrival time
        interarrival_time = np_random.exponential(1.0 / arrival_rate)
        yield env.timeout(interarrival_time)

        # Deterministic service time (1 / service rate)
        service_time = 1.0 / service_rate
        arrival_time = env.now

        def service(env, service_time, arrival_time):
            with servers.request() as request:
                # Wait for a server to be available
                yield request

                # Record waiting time
                wait_time = env.now - arrival_time
                waiting_time.append(wait_time)

                # Simulate the service time
                yield env.timeout(service_time)

        # Start the service process
        env.process(service(env, service_time, arrival_time))


def run_md_simulation(num_servers, arrival_rate, service_rate, sim_time, seed):
    """Run the M/D/1 or M/D/n simulation."""
    np_random = np.random.RandomState(seed)
    env = simpy.Environment()

    # List to store waiting times
    waiting_time = []

    # Create a resource to represent the servers
    servers = simpy.Resource(env, capacity=num_servers)
    env.process(arrival_process_md(env, servers, arrival_rate, service_rate, waiting_time, np_random))

    # Run the simulation
    env.run(until=sim_time)

    # Return the average waiting time
    return np.mean(waiting_time)


if __name__ == '__main__':
    print("\n--- M/D/1 and M/D/n Queues ---")
    for num_servers in NUM_SERVERS:
        replications = []

        for i in range(NUM_REPLICATIONS):
            seed = RANDOM_SEED + i
            avg_waiting_time = run_md_simulation(num_servers, ARRIVAL_RATE, SERVICE_RATE, SIM_TIME, seed)
            replications.append(avg_waiting_time)

        # Compute confidence interval
        mean, lower_bound, upper_bound = compute_confidence_interval(replications)

        print(f"\nNumber of Servers: {num_servers} (M/D/{num_servers})")
        print(f"Mean Waiting Time: {mean:.2f}")
        print(f"95% Confidence Interval: ({lower_bound:.2f}, {upper_bound:.2f})")



--- M/D/1 and M/D/n Queues ---

Number of Servers: 1 (M/D/1)
Mean Waiting Time: 2503.73
95% Confidence Interval: (2493.41, 2514.06)

Number of Servers: 3 (M/D/3)
Mean Waiting Time: 0.23
95% Confidence Interval: (0.23, 0.24)

Number of Servers: 2 (M/D/2)
Mean Waiting Time: 40.46
95% Confidence Interval: (32.12, 48.80)

Number of Servers: 4 (M/D/4)
Mean Waiting Time: 0.05
95% Confidence Interval: (0.05, 0.05)

Number of Servers: 5 (M/D/5)
Mean Waiting Time: 0.01
95% Confidence Interval: (0.01, 0.01)


In [67]:
def arrival_process_hyperexp(env, servers, arrival_rate, waiting_time, np_random):
    """Generate new jobs with hyperexponential service times."""
    while True:
        # Generate interarrival time
        interarrival_time = np_random.exponential(1.0 / arrival_rate)
        yield env.timeout(interarrival_time)

        # Hyperexponential service time
        if np_random.uniform() < 0.75:  # 75% jobs with avg service time = 1.0
            service_time = np_random.exponential(1.0)
        else:  # 25% jobs with avg service time = 5.0
            service_time = np_random.exponential(5.0)

        arrival_time = env.now

        def service(env, service_time, arrival_time):
            with servers.request() as request:
                # Wait for a server to be available
                yield request

                # Record waiting time
                wait_time = env.now - arrival_time
                waiting_time.append(wait_time)

                # Simulate the service time
                yield env.timeout(service_time)

        # Start the service process
        env.process(service(env, service_time, arrival_time))


def run_hyperexp_simulation(num_servers, arrival_rate, sim_time, seed):
    """Run the M/H2/1 or M/H2/n simulation."""
    np_random = np.random.RandomState(seed)
    env = simpy.Environment()

    # List to store waiting times
    waiting_time = []

    # Create a resource to represent the servers
    servers = simpy.Resource(env, capacity=num_servers)
    env.process(arrival_process_hyperexp(env, servers, arrival_rate, waiting_time, np_random))

    # Run the simulation
    env.run(until=sim_time)

    # Return the average waiting time
    return np.mean(waiting_time)


if __name__ == '__main__':
    print("\n--- M/H2/1 and M/H2/n Queues ---")
    for num_servers in NUM_SERVERS:
        replications = []

        for i in range(NUM_REPLICATIONS):
            seed = RANDOM_SEED + i
            avg_waiting_time = run_hyperexp_simulation(num_servers, ARRIVAL_RATE, SIM_TIME, seed)
            replications.append(avg_waiting_time)

        # Compute confidence interval
        mean, lower_bound, upper_bound = compute_confidence_interval(replications)

        print(f"\nNumber of Servers: {num_servers} (M/H2/{num_servers})")
        print(f"Mean Waiting Time: {mean:.2f}")
        print(f"95% Confidence Interval: ({lower_bound:.2f}, {upper_bound:.2f})")



--- M/H2/1 and M/H2/n Queues ---

Number of Servers: 1 (M/H2/1)
Mean Waiting Time: 3730.55
95% Confidence Interval: (3706.15, 3754.95)

Number of Servers: 3 (M/H2/3)
Mean Waiting Time: 1221.90
95% Confidence Interval: (1194.16, 1249.65)

Number of Servers: 2 (M/H2/2)
Mean Waiting Time: 2466.14
95% Confidence Interval: (2439.06, 2493.23)

Number of Servers: 4 (M/H2/4)
Mean Waiting Time: 63.93
95% Confidence Interval: (48.83, 79.03)

Number of Servers: 5 (M/H2/5)
Mean Waiting Time: 1.80
95% Confidence Interval: (1.69, 1.90)
